In [16]:
import tensorflow as tf
import numpy as np
import pickle
with open('sentiment_set.pickle','rb') as p:
    train_x, train_y, test_x, test_y = pickle.load(p)

In [17]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

In [18]:
# height * width
x = tf.placeholder('float',[None,len(train_x[0])])
y = tf.placeholder('float')

In [19]:
def neural_network_model(data):
    # input_data * weights + biases
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(train_x[0]), n_nodes_hl1])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']) , output_layer['biases'])
    
    return output

In [20]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # cycles feedback + backprops
    hm_epochs = 10
    
    with tf.Session() as alpha:
        alpha.run(tf.global_variables_initializer())
        
        # training the network
        for e in range(hm_epochs):
            epoch_loss = 0
            
            i=0
            while i<len(train_x):
                start=i
                end = i+batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
           
                _, c = alpha.run([optimizer,cost],feed_dict={x:batch_x, y:batch_y})
                epoch_loss +=c
                i+= batch_size
            print('Epoch {} completed out of {} loss: {}'.format(e,hm_epochs,epoch_loss))
        
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:test_x, y:test_y}))

In [21]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 265022.67578125
Epoch 1 completed out of 10 loss: 118037.59680175781
Epoch 2 completed out of 10 loss: 72467.92153930664
Epoch 3 completed out of 10 loss: 46196.748123168945
Epoch 4 completed out of 10 loss: 31987.741592407227
Epoch 5 completed out of 10 loss: 25183.302589416504
Epoch 6 completed out of 10 loss: 26375.66138458252
Epoch 7 completed out of 10 loss: 23732.98314666748
Epoch 8 completed out of 10 loss: 14219.028512954712
Epoch 9 completed out of 10 loss: 7942.633330345154
Accuracy:  0.625704
